In [1]:
import pandas as pd
from pathlib import Path

## Cleaning and exporting the data

We hve four sources of data that we need to clean and consolidate into one to be added in an posibble s3 file. 

In [2]:
files_types = {"cbk" : "csv", "fred" : "csv"}
# files_types = {"cbk" : "csv", "fred" : "csv", "m" : "xls", "x" : "xls"}

We read the data

In [3]:
raw_data = {}
for k, v in files_types.items():
    if v == "xls":
        raw_data[k] = pd.read_excel(Path.cwd().parent.joinpath("data", "raw_" + k + "." + v))
    else:
        raw_data[k] = pd.read_csv(Path.cwd().parent.joinpath("data", "raw_" + k + "." + v))

WARNING *** file size (87872) not 512 + multiple of sector size (512)
WARNING *** file size (87872) not 512 + multiple of sector size (512)


We explore the data

In [4]:
raw_data["cbk"]

,cod,date,value,var
0,1992,1999-01-31T00:00:00-06:00,528.559625,exp
1,1992,1999-02-28T00:00:00-06:00,1101.014371,exp
2,1992,1999-03-31T00:00:00-06:00,1686.540258,exp
3,1992,1999-04-30T00:00:00-06:00,2315.256577,exp
4,1992,1999-05-31T00:00:00-06:00,2869.498299,exp
...,...,...,...,...
53267,3799,2022-08-15T00:00:00-06:00,NaN,mil_count
53268,3799,2022-08-16T00:00:00-06:00,18.000000,mil_count
53269,3799,2022-08-17T00:00:00-06:00,18.000000,mil_count
53270,3799,2022-08-18T00:00:00-06:00,18.000000,mil_count


In [5]:
raw_data["fred"]

,date,value,var
0,1986-01-02,25.56,crude_oil_wti
1,1986-01-03,26.00,crude_oil_wti
2,1986-01-06,26.53,crude_oil_wti
3,1986-01-07,25.85,crude_oil_wti
4,1986-01-08,25.87,crude_oil_wti
...,...,...,...
103279,2022-08-16,3.01,treasury6month
103280,2022-08-17,3.03,treasury6month
103281,2022-08-18,3.01,treasury6month
103282,2022-08-19,3.04,treasury6month


In [6]:
raw_data["x"]

,Unnamed: 0,Unnamed: 1,Unnamed: 2,Unnamed: 3,Unnamed: 4,Unnamed: 5,Unnamed: 6,Unnamed: 7,Unnamed: 8,Unnamed: 9,...,Unnamed: 17,Unnamed: 18,Unnamed: 19,Unnamed: 20,Unnamed: 21,Unnamed: 22,Unnamed: 23,Unnamed: 24,Unnamed: 25,Unnamed: 26
0,NaN,1998.0,NaN,1999.0,2000.0,2001.0,2002.0,2003.0,2004.0,2005.0,...,2.013000e+03,2.014000e+03,2.015000e+03,2.016000e+03,2.017000e+03,2.018000e+03,2.019000e+03,2.020000e+03,2.021000e+03,2022*
1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,REGIMEN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,Definitivo,0.0,NaN,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,5.036826e+06,5.352381e+06,4.967463e+06,5.075730e+06,5.347056e+06,5.416194e+06,5.235324e+06,5.118868e+06,5.966781e+06,1615366.734036
4,Perfeccionamiento Activo,0.0,NaN,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,2.170279e+05,1.433158e+05,1.168891e+05,1.311091e+05,1.571880e+05,1.443899e+05,1.544929e+05,1.388538e+05,2.491643e+05,67777.64839
5,Zona Franca,0.0,NaN,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,3.345952e+06,3.671282e+06,4.154194e+06,4.715010e+06,5.109536e+06,5.668595e+06,5.974035e+06,6.371646e+06,8.270916e+06,2191664.806911
6,Grand Total,0.0,NaN,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,8.599806e+06,9.166979e+06,9.238546e+06,9.921850e+06,1.061378e+07,1.122918e+07,1.136385e+07,1.162937e+07,1.448686e+07,3874809.189337


Let's make 